### 1) Import the necessary headers

In [138]:
#NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#TensorFlow
import numpy as np
import tensorflow as tf
import tflearn 
import random

### 2) Import the intents file

In [139]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

### 3.1) Organize documents, words and classification classes

In [140]:
words = []
classes = []
documents = []
ignore_words = ['?']

#loop through each sentence in our intent patterns:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize word in each sentence
        w = nltk.word_tokenize(pattern)
        #add to our word list
        words.extend(w)
        #add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

### 3.2) Stem and Lower each word and remove duplicates

In [141]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]

In [142]:
words = sorted(list(set(words)))

In [143]:
classes = sorted(list(set(classes)))

In [144]:
print(len(documents),"documents",documents)
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

31 documents [(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['What', 'hours', 'are', 'you', 'open', '?'], 'hours'), (['What', 'are', 'your', 'hours', '?'], 'hours'), (['When', 'are', 'you', 'open', '?'], 'hours'), (['Which', 'mopeds', 'do', 'you', 'have', '?'], 'mopeds'), (['What', 'kinds', 'of', 'mopeds', 'are', 'there', '?'], 'mopeds'), (['What', 'do', 'you', 'rent', '?'], 'mopeds'), (['What', 'should', 'I', 'eat', 'today', '?'], 'food'), (['I', 'am', 'feeling', 'hungry', '.'], 'food'), (['What', 'is', 'in', 'the', 'menu', '?'], 'food'), (['I', 'want', 'to', 'eat', 'food'], 'food'), (['Do', 'you', 'take', 'credit', 'cards', '?'], 'payments'), (['Do', 'you', 'accept', 'Mastercard', '?'], 'paymen

### 4) Now this data needs to be converted for use by Tensorflow

In [145]:
#create training data

In [146]:
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [147]:
#training set, bag of words for each sentence

In [148]:
for doc in documents:     #document is a pair (x,y), where x is a word and y is the intent
    #initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    #stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    #create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else  bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

### 5) Shuffle our features and turn into np array

In [149]:
random.shuffle(training)
training = np.array(training)

In [150]:
#create train and test sets

train_x = list(training[:,0])
train_y = list(training[:,1])

### 6) Building the model

In [151]:
#reset underlying graph data
tf.reset_default_graph()
#Build Neural Network
net = tflearn.input_data(shape=[None,len(train_x[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net,tensorboard_dir='tflearn_logs')
# Start training (apply Gradient Descent)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.24152 | time: 0.021s
| Adam | epoch: 1000 | loss: 0.24152 - acc: 0.9806 -- iter: 24/31
Training Step: 4000  | total loss: 0.23173 | time: 0.026s
| Adam | epoch: 1000 | loss: 0.23173 - acc: 0.9825 -- iter: 31/31
--
INFO:tensorflow:/home/subhag/Projects/ContextChat/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


### 7) Save all the data structures

In [152]:
import pickle
pickle.dump({'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ))

### 8) Building the chatbot framework

In [153]:
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [97]:
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /home/subhag/Projects/ContextChat/model.tflearn


### 9) Before moving further, convert user input to Bag of Words

In [98]:
def clean_up_sentence(sentence):
    #tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    #stem each word
    sentence_words = [stemmer.stem(w.lower()) for w in sentence_words]
    return sentence_words

In [99]:
def bag_of_words(sentence, words, show_details=False):
    #tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    #bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if s == w:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" %w)
                    
    return(np.array(bag))

In [100]:
p = bag_of_words("I am hungry",words)

In [101]:
print(p)

[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### 10) Buillding the response processor

In [102]:
error_threshold = 0.25


def classify(sentence):
    #generate probabilities from the model
    results = model.predict([bag_of_words(sentence,words)])[0]
    #filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>error_threshold]
    #sort by strength of probability
    results.sort(key = lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]],r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

    results.pop(0)

In [113]:
response(sentence='Give me something to eat',show_details=True)

Try out some sandwhiches


### 11) Contextualization

In [132]:
# create a data structure to hold user context
error_threshold = 0.25
context = {}

def classify(sentence):
    #generate probabilities from the model
    results = model.predict([bag_of_words(sentence,words)])[0]
    #filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>error_threshold]
    #sort by strength of probability
    results.sort(key = lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]],r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123',show_details='False'):
    results = classify(sentence)
    #if we have a classification, find the matching intent tag
    if results:
        while results:
            for i in intents['intents']:
                #find a tag matching with first result
                if i['tag'] == results[0][0]:
                    #set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print('context:', i['context_set'])
                        context[userID] = i['context_set']

                        # check if this intent is contextual and applies to this user's conversation
                        if not 'context_filter' in i or \
                            (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                            if show_details: print ('tag:', i['tag'])
                            # a random response from the intent
                            return print(random.choice(i['responses']))

            results.pop(0)
            
            

In [137]:
response('I want a bike today')

context: rentalday
tag: rental
Are you looking to rent today or later this week?
